In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display
import pandas as pd
import spot
import sys
import re
spot.setup(show_default='.a')
pd.options.display.float_format = '{: .0f}'.format
pd.options.display.latex.multicolumn_format = 'c'
from evaluation_utils import split_cols, high_min, high_max, fix_type, fix_tool

In [2]:
rerun = False

In [3]:
%%bash
ltl3ba -v
ltl3tela -v
ltl2tgba --version
delag --version
ltl2dgra --version # Rabinizer 4

LTL3BA 1.1.3
LTL3TELA 1.2.1 (using Spot 2.7)
ltl2tgba (spot) 2.7

Copyright (C) 2018  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Name: owl
Version: 18.06
Name: owl
Version: 18.06


In [4]:
def generate(n=1000,func=(lambda x: True),filename=None,priorities='',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

In [5]:
f_rand = 'formulae/atva19/rand.ltl'
f_patterns = 'formulae/atva19/patterns.ltl'
# generate(1000, filename = f_rand)

### Deterministic automata

In [6]:
d_tools = {
    "ltl3tela-D1": "ltl3tela -D1 -f %f > %O",
    "ltl2tgba-DG": "ltl2tgba -DG %f > %O",
    "delag": "delag %f > %O",
    "rabinizer4": "ltl2dgra %f > %O"
}
d_order = ["ltl3tela-D1", "ltl2tgba-DG", "delag", "rabinizer4"]
d_cols = ["states", "edges", "acc"]

In [7]:
d_csv_rand = 'formulae/atva19/det.rand.csv'
d_data_rand = LtlcrossRunner(d_tools, formula_files = [f_rand], res_filename = d_csv_rand, cols = d_cols)
if rerun:
    d_data_rand.run_ltlcross(automata = False, timeout = '60')
d_data_rand.parse_results()

In [8]:
det_rand = d_data_rand.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_rand

column,states,edges,acc
tool,,,
ltl3tela-D1,5934,18520,1268
ltl2tgba-DG,6799,24131,1575
delag,7176,71672,3089
rabinizer4,7581,31099,2786


In [9]:
d_csv_patterns = 'formulae/atva19/det.patterns.csv'
d_data_patterns = LtlcrossRunner(d_tools, formula_files = [f_patterns], res_filename = d_csv_patterns, cols = d_cols)
if rerun:
    d_data_patterns.run_ltlcross(automata = False, timeout = '60')
d_data_patterns.parse_results()

In [10]:
det_err = pd.DataFrame(d_data_patterns.get_error_count(),columns=['err.literature'])
det_lit = d_data_patterns.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_lit = pd.concat([det_lit,det_err],axis=1,join='inner')
det_lit

,states,edges,acc,err.literature
tool,,,,
ltl3tela-D1,2536,10641,454,39
ltl2tgba-DG,3905,26643,652,20
delag,8661,2209807,1196,11
rabinizer4,2969,12358,1133,12


In [11]:
#to = d_data_patterns.exit_status
#to[to == "timeout"].count()
#len(d_data_patterns.values.dropna().index)

In [12]:
d_data_patterns.smaller_than('ltl3tela-D1', 'ltl2tgba-DG')

column                                                          states  \
tool                                                       ltl2tgba-DG   
form_id formula                                                          
50      G!p0 | F(p0 & (!p1 W p2))                                    5   
68      G((p0 & XFp1) -> XF(p1 & Fp2))                              10   
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...          19   
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...          16   
73      G(p0 -> F(p1 & XFp2))                                        6   
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          11   
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           7   
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  8   
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          81   
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              9   
310     Fp0 U Gp1                                                    3   
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   6   
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                5   
332     GFa1 U G(GFa0 U Xb)                                         21   
333     GFa1 U G(GFa0 U XXb)                                        22   
334     GFa1 U G(GFa0 U XXXb)                                       23   
335     GFa1 U G(GFa0 U XXXXb)                                      24   
336     GFa1 U G(GFa0 U XXXXXb)                                     25   
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                              257   
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             258   
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            259   
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           260   
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          261   
387     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U p4))))))              12   
388     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U (p4 & (p4...          51   

column                                                                  
tool                                                       ltl3tela-D1  
form_id formula                                                         
50      G!p0 | F(p0 & (!p1 W p2))                                    4  
68      G((p0 & XFp1) -> XF(p1 & Fp2))                               4  
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...           8  
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...           6  
73      G(p0 -> F(p1 & XFp2))                                        4  
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          10  
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           4  
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  7  
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          34  
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              7  
310     Fp0 U Gp1                                                    2  
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   4  
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                4  
332     GFa1 U G(GFa0 U Xb)                                         11  
333     GFa1 U G(GFa0 U XXb)                                        12  
334     GFa1 U G(GFa0 U XXXb)                                       13  
335     GFa1 U G(GFa0 U XXXXb)                                      14  
336     GFa1 U G(GFa0 U XXXXXb)                                     15  
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               11  
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                              12  
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                             13  
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                            14  
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                           15  
387     G(p1 -> (p1 U (p2 & (

In [13]:
d_data_patterns.smaller_than('ltl2tgba-DG', 'ltl3tela-D1')

Empty DataFrame
Columns: [(states, ltl2tgba-DG), (states, ltl3tela-D1)]
Index: []

In [14]:
det_tmp = pd.merge(det_rand, det_lit, suffixes=('.random','.literature'),on='tool')
det_tmp

,states.random,edges.random,acc.random,states.literature,edges.literature,acc.literature,err.literature
tool,,,,,,,
ltl3tela-D1,5934,18520,1268,2536,10641,454,39
ltl2tgba-DG,6799,24131,1575,3905,26643,652,20
delag,7176,71672,3089,8661,2209807,1196,11
rabinizer4,7581,31099,2786,2969,12358,1133,12


In [15]:
det = split_cols(det_tmp,'.').swaplevel(axis=1)
det

random              literature                   
            states  edges   acc     states    edges   acc err
tool                                                         
ltl3tela-D1   5934  18520  1268       2536    10641   454  39
ltl2tgba-DG   6799  24131  1575       3905    26643   652  20
delag         7176  71672  3089       8661  2209807  1196  11
rabinizer4    7581  31099  2786       2969    12358  1133  12

### Nondeterministic automata

In [16]:
import os
os.environ['SPOT_HOA_TOLERANT']='TRUE'

In [17]:
n_tools = {
    "ltl3tela": "ltl3tela -f %f > %O",
    "ltl2tgba": "ltl2tgba %f > %O",
    "ltl2tgba-G": "ltl2tgba -G %f > %O",
    "ltl3ba": "ltldo 'ltl3ba -H2' -f %f > %O",
}
n_order = ["ltl3tela", "ltl2tgba-G", "ltl2tgba", "ltl3ba"]
n_cols = ["states", "edges", "acc"]

In [18]:
n_csv_rand = 'formulae/atva19/nondet.rand.csv'
n_data_rand = LtlcrossRunner(n_tools, formula_files = [f_rand], res_filename = n_csv_rand, cols = n_cols)
if rerun:
    n_data_rand.run_ltlcross(automata = False, timeout = '60')
n_data_rand.parse_results()

In [19]:
nd_rand = n_data_rand.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]
nd_rand

column,states,edges,acc
tool,,,
ltl3tela,5109,12481,1135
ltl2tgba-G,5391,13144,1041
ltl2tgba,5413,13059,1034
ltl3ba,6103,15636,1616


In [20]:
n_csv_patterns = 'formulae/atva19/nondet.patterns.csv'

In [21]:
n_data_patterns = LtlcrossRunner(n_tools, formula_files = [f_patterns], res_filename = n_csv_patterns, cols = n_cols)
if rerun:
    n_data_patterns.run_ltlcross(automata = False, timeout = '60')
n_data_patterns.parse_results()

In [22]:
nd_err = pd.DataFrame(n_data_patterns.get_error_count(),columns=['err.literature'])
nd_lit = n_data_patterns.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]
nd_lit = pd.concat([nd_lit,nd_err],axis=1,join='inner')
nd_lit

,states,edges,acc,err.literature
tool,,,,
ltl3tela,2378,20718,544,28
ltl2tgba-G,2398,20555,642,12
ltl2tgba,2651,8721,502,11
ltl3ba,4654,21180,822,4


In [23]:
n_data_patterns.smaller_than('ltl3tela', 'ltl2tgba-G')

column                                                         states         
tool                                                       ltl2tgba-G ltl3tela
form_id formula                                                               
50      G!p0 | F(p0 & (!p1 W p2))                                   5        4
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...         10        8
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))               6        4
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               9        8
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             10        9
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            11       10
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           12       11
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          13       12
342     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U Xb)))                    11        9
343     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXb)))                   12       10
344     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXb)))                  13       11
345     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXb)))                 14       12
346     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXXb)))                15       13

In [24]:
n_data_patterns.smaller_than('ltl2tgba-G', 'ltl3tela')

Empty DataFrame
Columns: [(states, ltl2tgba-G), (states, ltl3tela)]
Index: []

In [25]:
nd_tmp = pd.merge(nd_rand, nd_lit, suffixes=('.random','.literature'),on='tool')
nd_tmp

,states.random,edges.random,acc.random,states.literature,edges.literature,acc.literature,err.literature
tool,,,,,,,
ltl3tela,5109,12481,1135,2378,20718,544,28
ltl2tgba-G,5391,13144,1041,2398,20555,642,12
ltl2tgba,5413,13059,1034,2651,8721,502,11
ltl3ba,6103,15636,1616,4654,21180,822,4


In [26]:
nd = split_cols(nd_tmp,'.').swaplevel(axis=1)
nd

random              literature                
           states  edges   acc     states  edges  acc err
tool                                                     
ltl3tela     5109  12481  1135       2378  20718  544  28
ltl2tgba-G   5391  13144  1041       2398  20555  642  12
ltl2tgba     5413  13059  1034       2651   8721  502  11
ltl3ba       6103  15636  1616       4654  21180  822   4

In [27]:
#rerun = True
#test_csv = 'formulae/atva19/test.csv'
#test_tools = {
#    'split-obligation': 'ltl3tela -D1 -f %f > %O',
#    'whole-obligation': 'ltl3tela -D1 -y1 -f %f > %O'
#}
#test_data = LtlcrossRunner(test_tools, formula_files = [f_patterns], res_filename = test_csv,
#                                 cols = ['states'])
#if not rerun:
#    test_data.run_ltlcross(timeout = '30')
#test_data.parse_results(test_csv)

In [28]:
#test_cols = ['states'];
#test_data.cummulative(col = test_cols).unstack(level = 0).loc[['split-obligation', 'whole-obligation'], test_cols]

In [29]:
n_data_patterns.get_error_count()

tool
ltl2tgba      11
ltl2tgba-G    12
ltl3ba         4
ltl3tela      28
dtype: int64

In [30]:
n_data_rand.get_error_count()

Series([], dtype: int64)

In [31]:
#test_data.smaller_than('split-obligation', 'whole-obligation')

### Merge tables


In [32]:
#Merge det & nondet
merged_tmp = pd.concat([det,nd],keys=["deterministic","nondeterministic"])

# Compute number of formulae
rand_lengths = f'({len(d_data_rand.values.dropna())}, {len(n_data_rand.values.dropna())})'
lit_lengths = f'({len(d_data_patterns.values.dropna())}, {len(n_data_patterns.values.dropna())})'

cols = merged_tmp.columns
# Add numbers to benchmark type
new_cols = []
for val in cols.get_level_values(0):
    val = val.replace('random',f'random {rand_lengths}')
    val = val.replace('literature',f'literature {lit_lengths}')
    new_cols.append(val)
# Replace with new index
new_index = pd.MultiIndex.from_tuples(zip(new_cols,cols.get_level_values(1)))
merged_tmp.columns = new_index
merged_tmp

random (1000, 1000)               \
                                          states  edges   acc   
                 tool                                           
deterministic    ltl3tela-D1                5934  18520  1268   
                 ltl2tgba-DG                6799  24131  1575   
                 delag                      7176  71672  3089   
                 rabinizer4                 7581  31099  2786   
nondeterministic ltl3tela                   5109  12481  1135   
                 ltl2tgba-G                 5391  13144  1041   
                 ltl2tgba                   5413  13059  1034   
                 ltl3ba                     6103  15636  1616   

                             literature (353, 368)                     
                                            states    edges   acc err  
                 tool                                                  
deterministic    ltl3tela-D1                  2536    10641   454  39  
                 ltl2tgba-DG                  3905    26643   652  20  
                 delag                        8661  2209807  1196  11  
                 rabinizer4                   2969    12358  1133  12  
nondeterministic ltl3tela                     2378    20718   544  28  
                 ltl2tgba-G                   2398    20555   642  12  
                 ltl2tgba                     2651     8721   502  11  
                 ltl3ba                       4654    21180   822   4

#### Highlight

In [33]:
det_high = det.astype(float).apply(high_min)

In [34]:
tmp_d = merged_tmp.loc['deterministic'].astype(float).apply(high_min)
tmp_n = merged_tmp.loc['nondeterministic'].astype(float).apply(high_min)
merged = pd.concat([tmp_d,tmp_n],keys=["deterministic","nondeterministic"])

In [35]:
merged

random (1000, 1000)                               \
                                          states          edges           acc   
                 tool                                                           
deterministic    ltl3tela-D1        \high $5934$  \high $18520$  \high $1268$   
                 ltl2tgba-DG                6799          24131          1575   
                 delag                      7176          71672          3089   
                 rabinizer4                 7581          31099          2786   
nondeterministic ltl3tela           \high $5109$  \high $12481$          1135   
                 ltl2tgba-G                 5391          13144          1041   
                 ltl2tgba                   5413          13059  \high $1034$   
                 ltl3ba                     6103          15636          1616   

                             literature (353, 368)                 \
                                            states          edges   
                 tool                                               
deterministic    ltl3tela-D1          \high $2536$  \high $10641$   
                 ltl2tgba-DG                  3905          26643   
                 delag                        8661        2209807   
                 rabinizer4                   2969          12358   
nondeterministic ltl3tela             \high $2378$          20718   
                 ltl2tgba-G                   2398          20555   
                 ltl2tgba                     2651   \high $8721$   
                 ltl3ba                       4654          21180   

                                                       
                                      acc         err  
                 tool                                  
deterministic    ltl3tela-D1  \high $454$          39  
                 ltl2tgba-DG          652          20  
                 delag               1196  \high $11$  
                 rabinizer4          1133          12  
nondeterministic ltl3tela             544          28  
                 ltl2tgba-G           642          12  
                 ltl2tgba     \high $502$          11  
                 ltl3ba               822   \high $4$

In [36]:
ci = [(fix_type(t[0],True,None),fix_tool(t[1])) for t in merged.index.values]
merged.index=pd.MultiIndex.from_tuples(ci)
merged

random (1000, 1000)  \
                                                                                        states   
\rotatebox[origin=c]{90}{\footnotesize det.}    \spottool{ltl3tela -D1}           \high $5934$   
                                                \spottool{ltl2tgba -DG}                   6799   
                                                Delag                                     7176   
                                                Rabinizer 4\tgdra                         7581   
\rotatebox[origin=c]{90}{\footnotesize nondet.} \spottool{ltl3tela}               \high $5109$   
                                                \spottool{ltl2tgba -G}                    5391   
                                                \spottool{ltl2tgba\tgba}                  5413   
                                                \spottool{ltl3ba -H2\tgba}                6103   

                                                                                           \
                                                                                    edges   
\rotatebox[origin=c]{90}{\footnotesize det.}    \spottool{ltl3tela -D1}     \high $18520$   
                                                \spottool{ltl2tgba -DG}             24131   
                                                Delag                               71672   
                                                Rabinizer 4\tgdra                   31099   
\rotatebox[origin=c]{90}{\footnotesize nondet.} \spottool{ltl3tela}         \high $12481$   
                                                \spottool{ltl2tgba -G}              13144   
                                                \spottool{ltl2tgba\tgba}            13059   
                                                \spottool{ltl3ba -H2\tgba}          15636   

                                                                                          \
                                                                                     acc   
\rotatebox[origin=c]{90}{\footnotesize det.}    \spottool{ltl3tela -D1}     \high $1268$   
                                                \spottool{ltl2tgba -DG}             1575   
                                                Delag                               3089   
                                                Rabinizer 4\tgdra                   2786   
\rotatebox[origin=c]{90}{\footnotesize nondet.} \spottool{ltl3tela}                 1135   
                                                \spottool{ltl2tgba -G}              1041   
                                                \spottool{ltl2tgba\tgba}    \high $1034$   
                                                \spottool{ltl3ba -H2\tgba}          1616   

                                                                           literature (353, 368)  \
                                                                                          states   
\rotatebox[origin=c]{90}{\footnotesize det.}    \spottool{ltl3tela -D1}             \high $2536$   
                                                \spottool{ltl2tgba -DG}                     3905   
                                                Delag                                       8661   
                                                Rabinizer 4\tgdra                           2969   
\rotatebox[origin=c]{90}{\footnotesize nondet.} \spottool{ltl3tela}                 \high $2378$   
                                                \spottool{ltl2tgba -G}                      2398   
                                                \spottool{ltl2tgba\tgba}                    2651   
                                                \spottool{ltl3ba -H2\tgba}                  4654   

                                                                                           \
                                                                                    edges   
\rotatebox[origin=c]{90}{\footnotesize det.}    \spotto

In [37]:
filename = 'colored_res.tex'

In [38]:
def cummulative_to_latex(res,file,transpose=False,color=True):
    if transpose:
        res = res.T
    col_f = 'cr'
    color_type = 'a' if color else 'r'
    cols = res.columns
    # Iterate over outermost-columns
    # The multiindex is a tuple:
    #  * levels [list of lists of column names]
    #  * labels [list of lists of pointers to levels] (the inner
    #                           lists have size = number of columns)
    # Each column col on level lev is labeled in the table by the
    # name stored in levels[lev][labels[lev][col]]
    lab = 0
    i = 0
    while lab < len(cols.labels[0]):
        # How many columns do we have for the current outermost-level column
        label_index = cols.labels[0][lab]
        level_length = len(res[cols.levels[0][label_index]].columns)
        # color if even
        if i % 2 == 0:
            col_f += color_type*level_length
        else:
            col_f += 'r'*level_length
        # i for colors, lab for iterating over outermost-level columns
        i += 1
        lab += level_length
    
    res.to_latex(buf=open(file,'w'), multirow=True,
         escape=False, na_rep='---',
         float_format=lambda x: '$' + '%0.0f' % x + '$',
         column_format=col_f, multicolumn_format='c')
    if color:
        color_table(file,extrarowheight='.2ex')

In [39]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

In [40]:
def color_table(filename, extrarowheight='.75ex'):
    setup = '''\\newcolumntype{{a}}{{>{{\\columncolor{{blue!10}}}}r}}
\\newcolumntype{{b}}{{>{{\\columncolor{{blue!10}}}}c}}
\\setlength{{\\aboverulesep}}{{0pt}}
\\setlength{{\\belowrulesep}}{{0pt}}
\\setlength{{\\extrarowheight}}{{{}}}
\\setlength{{\\heavyrulewidth}}{{.8pt}}
\\def\\high{{\\cellcolor{{green!40}}}}
\\setlength\\tabcolsep{{.8em}}
'''.format(extrarowheight)
    with open(filename) as f:
        lines = f.read()
    lines = fix_header_colors(lines)
    with open(filename,"w") as f1:
        f1.write(setup + lines)

In [41]:
def make_heading(filename):
    with open(filename) as f:
        lines = f.read()
    lines = re.sub(r"\s+&\s+&\s+states", 
        r"& tool & states",
        lines)
    lines = re.sub(r"\{\}\s+&\s+tool\s+&\s+&.*\\\\", 
        r"%",
        lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [42]:
def fix_lines(filename, end=9, vertical=False):
    end = str(end)
    with open(filename) as f:
        lines = f.read()
    lines = lines.replace('cline','cmidrule')
    lines = lines.replace('$nan$','---')
    with open(filename,"w") as f1:
        f1.write(lines)

In [43]:
cummulative_to_latex(merged,filename)
make_heading(filename)
fix_lines(filename, 9)

In [44]:
cp {filename} /home/xblahoud/research/ltl3tela_toolpaper/